In [ ]:
import torch
import numpy as np
import time

In [283]:
in_image = torch.randn(12,64,64)
nn = torch.nn.Conv2d(12, 3, 3, padding=3, stride=2)

In [284]:
out_image = nn(in_image)

In [285]:
out_channels = nn.out_channels
in_channels = nn.in_channels
in_height = in_image.size(1)
in_width = in_image.size(2)
kernel_size = nn.kernel_size[0]
out_height = out_image.size(1)
out_width = out_image.size(2)
weights = nn.weight.detach()
stride = nn.stride[0]
padding = nn.padding[0]
bias = nn.bias
print(weights.size())

torch.Size([3, 12, 3, 3])


In [ ]:
weights.shape

In [ ]:
now=time.time()

padded_size = list(in_image.size())
padded_size = [ub_mult.size(0)] + padded_size
padded_size[2] = padded_size[2] + 2 * padding
padded_size[3] = padded_size[3] + 2 * padding

ub_mult = torch.reshape(ub_mult, (ub_mult.size(0),out_channels, out_height, out_width))
im = torch.zeros(padded_size)
print(im.size())
print(ub_mult.size())

for i_out in range(out_channels):
    channel_weights = weights[i_out]
    print(channel_weights.size())
    for i_h in range(out_height):
        for i_w in range(out_width):
            el = ub_mult[:,i_out,i_h,i_w]
            print(el.size())
            im[:,:,stride*i_h:stride*i_h+kernel_size, stride*i_w:stride*i_w+kernel_size] += el.view(-1,1,1,1) * channel_weights
            
if padding > 0:
    im = im[:,padding:-padding,padding:-padding]
print(time.time()-now)


In [ ]:
ub_mult.shape

sum=ub_mult.sum(dim=[2,3])
bias_r = bias.view(1,-1)
(sum * bias_r).sum(1)



In [288]:

now=time.time()
in_width_p = in_width + padding * 2
in_height_p = in_height + padding * 2

size_p = in_height_p * in_width_p
in_dim = size_p * in_channels
out_dim = out_height * out_width * out_channels
res = torch.zeros((out_dim, in_dim))

# build row fillers
len_rows = (in_channels - 1) * size_p + (kernel_size - 1) * in_width_p + kernel_size
channels = torch.zeros((out_channels, len_rows))
print(channels.size())

# for each out_channel build a row of the transpose matrix that corresponds to the weights part (0s are cut away) these weights correspond to a single output pixel (and in shifted form they correspond to a whole out_channel)
for i_out in range(out_channels):
    for i_in in range(in_channels):
        for k in range(kernel_size):
            # fill zero row with kernel_size values at a time corresponding to a application of a row of kernel weights a kernel_size many consecutive inputs
            start = i_in * size_p + k * in_width_p
            end = start + kernel_size
            channels[i_out, start:end] = weights[i_out, i_in, k]
            
    # now need to generate actual rows from this channelrow by putting it at shifted locations
    for i_out_height in range(out_height):
        for i_out_width in range(out_width):
            # for width many outputs we need to shift the channel by a stride (since thats how conv kernel gets applied by shifting rowwise
            # and we need to shift the channel by a whole row of the input image (since thats how conv kernel gets applied by shifting rowwise)
            start = i_out_height * stride * in_width_p + i_out_width * stride
            end = start + len_rows
            # the output pixel is at the position of the channel row in the output matrix: for one step in outchannels we skip a whole out_height * out_width, for one step in out_height we skip out_width many, for one step in out_width we skip one
            output = i_out * out_height * out_width + i_out_height * out_width + i_out_width
            res[output, start:end] = channels[i_out]
# certain cols correspond to padding pixels, we need to delete them

if padding !=0:
    ones=torch.ones_like(in_image)
    mask=torch.zeros([in_channels,in_width_p,in_width_p])
    mask[:,padding:-padding,padding:-padding]=ones
    mask=mask.flatten()
    res=res[:,torch.tensor(mask)==1]

if bias is None:
    ret_bias = torch.zeros(self.out_size)
else:
    ret_bias = torch.repeat_interleave(bias, out_width * out_height)


torch.Size([3, 54043])
0.8237879276275635


In [287]:
res

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0217,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0023, -0.0459,  0.0217],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0023]])

In [289]:
lc

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0217,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0023, -0.0459,  0.0217],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0023]])

In [278]:
res.size()

torch.Size([75, 25])

In [223]:

def row_toeplitz(row):
    col = np.zeros_like(row)
    col[0] = row[0]

    vals = np.concatenate((col[::-1], row[1:]))
    out_shp = len(row), len(row)
    n = vals.strides[0]

    return np.lib.stride_tricks.as_strided(vals[len(col)-1:], shape=out_shp, strides=(-n, n)).copy()
now=time.time()
concat_weights = np.zeros((1,in_width**2 * in_channels))  
print(concat_weights)
for i in range(out_channels):
    # loop over output channels and then append to the lower side

    weights_per_in_channel = []
    for j in range(in_channels):
        # loop over input channels and then append to the right side
        # print(i,j)
            
        kernel_mat = weights[i,j,:,:]
        idx = np.arange((in_width + 2*padding)**2)

        # cut away first rows of padding
        cond1 = idx < (in_width + 2*padding) * padding

        # cut away lower row of padding
        cond2 = idx >= (in_width + 2*padding) * (in_width + padding)

        # cut away left columns of padding
        cond3 = idx % (in_width + 2*padding) < padding

        # cut away right columns of padding
        cond4 = idx % (in_width + 2*padding) >= (in_width+ padding)

        remove_padding = np.logical_or.reduce(
            (cond1, cond2, cond3, cond4)
        )

        # right columns not considered by kernel
        cond5 = idx % (in_width + 2*padding) > ((in_width+ 2*padding) - kernel_size)

        # lower rows not considered by kernel
        cond6 = idx >= (in_width + 2*padding) * (in_width + 2*padding - kernel_size + 1)



        if stride > 0:
            cond7 = idx % (in_width + 2*padding) % stride != 0
            cond8 = idx // (in_width + 2*padding)  % stride != 0
        else:
            cond7 = np.zeros_like(cond1).astype(bool)
            cond8 = np.zeros_like(cond1).astype(bool)

        # if row used by anyone (row is corresp to an output value then)
        remove_rows = np.logical_or.reduce(
            (cond5, cond6, cond7, cond8)
        )
        
        row_toep = np.vstack(
            (
                np.hstack(
                    (
                        kernel_mat, 
                        np.zeros(
                            (
                                kernel_size, 
                                in_width + 2*padding - kernel_size
                            )
                        )
                    )
                ),
                np.zeros(
                    (
                        in_width + 2*padding - kernel_size, 
                        in_width + 2*padding
                    )
                )
            )
        ).reshape(-1)
        
        toep_mat = row_toeplitz(row_toep)
        weights_1 = toep_mat[~remove_rows, :][:, ~remove_padding]
        print(weights_1.shape)
        weights_per_in_channel.append(weights_1)

    if in_channels > 1:
        # first concatenate all the weights -> direction before we append it to np array
        concat_weights = np.vstack((
            concat_weights, 
            np.concatenate(tuple(weights_per_in_channel), axis=1)
        ))
    else:
        # we have to write the loop otherwise concatenate will fail
        
        concat_weights = np.vstack((concat_weights, weights_1))
    print(concat_weights.shape)
a=torch.tensor(concat_weights[1:,:], dtype = torch.float32)


[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]]
(9, 25)
(10, 25)
(9, 25)
(19, 25)
(9, 25)
(28, 25)


In [224]:
a


tensor([[ 0.0520,  0.0188,  0.0669,  0.0000,  0.0000,  0.3322,  0.1600, -0.2262,
          0.0000,  0.0000,  0.0053,  0.0103, -0.0601,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000],
        [ 0.0000,  0.0520,  0.0188,  0.0669,  0.0000,  0.0000,  0.3322,  0.1600,
         -0.2262,  0.0000,  0.0000,  0.0053,  0.0103, -0.0601,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000],
        [ 0.0000,  0.0000,  0.0520,  0.0188,  0.0669,  0.0000,  0.0000,  0.3322,
          0.1600, -0.2262,  0.0000,  0.0000,  0.0053,  0.0103, -0.0601,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0520,  0.0188,  0.0669,
          0.0000,  0.0000,  0.3322,  0.1600, -0.2262,  0.0000,  0.0000,  0.0053,
          0.0103, -0.0601,  0.0000,  0.0000,  0.0000

In [213]:
concat_weights[1:,:]

array([[ 0.05202016,  0.01877633,  0.06692675,  0.        ,  0.        ,
         0.33216378,  0.15995803, -0.22615787,  0.        ,  0.        ,
         0.00531805,  0.01030552, -0.06007913,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.05202016,  0.01877633,  0.06692675,  0.        ,
         0.        ,  0.33216378,  0.15995803, -0.22615787,  0.        ,
         0.        ,  0.00531805,  0.01030552, -0.06007913,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.05202016,  0.01877633,  0.06692675,
         0.        ,  0.        ,  0.33216378,  0.15995803, -0.22615787,
         0.        ,  0.        ,  0.00531805,  0.01030552, -0.06007913,
         0.        ,  0.        ,  0.        ,  0

In [214]:
torch.tensor(concat_weights[1:,:], dtype = torch.float32)

tensor([[ 0.0520,  0.0188,  0.0669,  0.0000,  0.0000,  0.3322,  0.1600, -0.2262,
          0.0000,  0.0000,  0.0053,  0.0103, -0.0601,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000],
        [ 0.0000,  0.0520,  0.0188,  0.0669,  0.0000,  0.0000,  0.3322,  0.1600,
         -0.2262,  0.0000,  0.0000,  0.0053,  0.0103, -0.0601,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000],
        [ 0.0000,  0.0000,  0.0520,  0.0188,  0.0669,  0.0000,  0.0000,  0.3322,
          0.1600, -0.2262,  0.0000,  0.0000,  0.0053,  0.0103, -0.0601,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0520,  0.0188,  0.0669,
          0.0000,  0.0000,  0.3322,  0.1600, -0.2262,  0.0000,  0.0000,  0.0053,
          0.0103, -0.0601,  0.0000,  0.0000,  0.0000

In [204]:
a

tensor([[ 0.2194, -0.3077,  0.0883,  0.0000,  0.0000,  0.3214, -0.1539,  0.0251,
          0.0000,  0.0000, -0.0775, -0.2331,  0.2016,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000],
        [ 0.0000,  0.2194, -0.3077,  0.0883,  0.0000,  0.0000,  0.3214, -0.1539,
          0.0251,  0.0000,  0.0000, -0.0775, -0.2331,  0.2016,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000],
        [ 0.0000,  0.0000,  0.2194, -0.3077,  0.0883,  0.0000,  0.0000,  0.3214,
         -0.1539,  0.0251,  0.0000,  0.0000, -0.0775, -0.2331,  0.2016,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.2194, -0.3077,  0.0883,
          0.0000,  0.0000,  0.3214, -0.1539,  0.0251,  0.0000,  0.0000, -0.0775,
         -0.2331,  0.2016,  0.0000,  0.0000,  0.0000

In [215]:
concat_weights[1:,:]

array([[ 0.05202016,  0.01877633,  0.06692675,  0.        ,  0.        ,
         0.33216378,  0.15995803, -0.22615787,  0.        ,  0.        ,
         0.00531805,  0.01030552, -0.06007913,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.05202016,  0.01877633,  0.06692675,  0.        ,
         0.        ,  0.33216378,  0.15995803, -0.22615787,  0.        ,
         0.        ,  0.00531805,  0.01030552, -0.06007913,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.05202016,  0.01877633,  0.06692675,
         0.        ,  0.        ,  0.33216378,  0.15995803, -0.22615787,
         0.        ,  0.        ,  0.00531805,  0.01030552, -0.06007913,
         0.        ,  0.        ,  0.        ,  0

In [ ]:
# create zero matirx of size corrwsponign